In [2]:
import torch

### 1. Pytorch 自动求导数

In [37]:
## 1. Inference
x= torch.ones(2,2,requires_grad = True)
print("*"*50,'\n x is : \n',x)

y = x + 2
print("*"*50,'\n y = x + 2 is :\n',y)

z = y * y  * 3
print("*"*50,'\n z = y * y *3 is :\n',z)

out = z.mean()
print("*"*50, '\n out = y.mean() is :\n', out)

## 2 . Backward - out is scalar
out.backward()
print("\n","*"*50, "\n x grad is:\n", x.grad)

************************************************** 
 x is : 
 tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
************************************************** 
 y = x + 2 is :
 tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
************************************************** 
 z = y * y *3 is :
 tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
************************************************** 
 out = y.mean() is :
 tensor(27., grad_fn=<MeanBackward0>)

 ************************************************** 
 x grad is:
 tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [48]:
## when output is vector 
x = torch.randn(3, requires_grad = True)
y = x * 2
while y.data.norm() < 1000:
    y  = y * 2
print('\n',x, '\n',y)
v = torch.tensor([1,1,.5])
y.backward(v)
print(x.grad)


 tensor([-0.5289, -2.7384,  0.5890], requires_grad=True) 
 tensor([ -270.8058, -1402.0596,   301.5794], grad_fn=<MulBackward0>)
tensor([512., 512., 256.])


### 2. Pytorch: 定义新的autograd函数

In [50]:
import torch
from torch.autograd import Variable

class MyRelu(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min = 0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

        

SyntaxError: unexpected EOF while parsing (<ipython-input-50-049b5772965b>, line 10)